In [1]:
import json

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 20)

In [3]:
import utils.checker as check
import utils.presenter as show
import utils.schedule_solver__with_arrivals as ssv

import utils.converter as convert

In [4]:
import utils.rolling_planning as rp

In [5]:
from ProductionDaySimulation import ProductionDaySimulation

In [6]:
import utils.schedule_solver__rescheduling as re_ssv

In [7]:
# Datei laden
with open("data/jobshop_instances.json", "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]
show.print_jobs(instance)

job 0:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
job 1:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
job 2:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
job 3:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
job 4:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
job 5:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
job 6:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
job 7:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
job 8:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
job 9:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 45]]



## Day 0

In [9]:
jssp_data, df_times = rp.init_jobs_with_arrivals(instance, 3, u_b_mmax = 0.75, 
                                                    generate_deadlines = True, deadlines_buffer_factor= 1.9)
show.print_jobs(jssp_data)

Job_000:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_001:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_002:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_003:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_004:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_005:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_006:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_007:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_008:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_009:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 4

In [10]:
df_times 

,Job,Arrival,Deadline
0,Job_000,14.37,2362.556555
1,Job_001,116.28,3148.115805
2,Job_002,141.78,3518.412818
3,Job_003,192.34,4086.168337
4,Job_004,236.45,2572.747002
...,...,...,...
64,Job_064,3749.15,6085.447002
65,Job_065,3907.00,7111.234311
66,Job_066,4061.64,6409.826555
67,Job_067,4110.12,7141.955805


### 1) Scheduling

In [ ]:
minutes = 45
df_plan0, total = ssv.solve_jssp_weighted_individual_flowtime(jssp_data, df_times, solver_time_limit= 60*minutes, epsilon=0.06)
check.check_all_constraints(df_plan0, jssp_data)

In [ ]:
print(f"Globale Makespan:{max(df_plan0.End)}")
df_plan0

In [ ]:
df = df_plan0

block = 1440                                        
for idx, part_df in df.groupby(df['Arrival'] // block, sort=True):
    show.plot_gantt_machines(part_df, f"Gantt-Diagramm Arrival-Tag {int(idx)}")

### 2) Simulation für Tag 0

In [ ]:
day_start = 0
day_end = 1440

filt = (df_plan0["Start"] >=  day_start) & (df_plan0["Start"] < day_end)
df_schedule_0 = df_plan0[filt]
df_schedule_0

In [ ]:
simulation = ProductionDaySimulation(df_schedule_0, vc=0.25)
df_execution, df_undone = simulation.run(end_time=day_end)

In [ ]:
df_execution

In [ ]:
show.plot_gantt_machines(df_execution, duration_column = "Simulated Processing Time")

### 3) Verbliebe Jobs nach Tag 0

In [ ]:
df_plan = df_plan0

df_rest = df_plan.merge(df_execution[["Job", "Machine"]], on=["Job", "Machine"], how="left", indicator=True)
df_rest = df_rest[df_rest["_merge"] == "left_only"].drop(columns=["_merge"])
df_rest = df_rest.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_rest

In [ ]:
jssp_data_rest = convert.get_jssp_from_schedule(df_rest)
show.print_jobs(jssp_data_rest)

In [ ]:
allowed = df_rest["Job"].unique() 
df_time_rest = df_times[df_times["Job"].isin(allowed)].copy()
df_time_rest

## Day 1

In [ ]:
jssp_data1, df_times1 = rp.update_new_day(jssp_data_rest, df_time_rest, instance, u_b_mmax= 0.75, generate_deadlines = True, deadlines_buffer_factor= 1.9)
show.print_jobs(jssp_data1)
df_times1

### 1) Rescheduling

In [ ]:
minutes = 45

In [ ]:
# Reduktion der bereits ausgeführten Operationen auf die relevanten für die nächsten Tage
df_execution_important = df_execution[df_execution["End"] >= 1440]
df_execution_important


df_plan1 = re_ssv.solve_jssp_weighted_with_fixed_operations_fast(
    job_dict=jssp_data1,
    df_arrivals=df_times1,
    df_executed=df_execution_important,
    solver_time_limit=60*minutes,
    epsilon=0.6,
    arrival_column="Arrival",
    reschedule_start=1440
)

In [ ]:
print(f"Final End:{max(df_plan1.End)}")
df_plan1

In [ ]:
df = df_plan1

block = 1440                                        
for idx, part_df in df.groupby(df['Arrival'] // block, sort=True):
    show.plot_gantt_machines(part_df, f"Gantt-Diagramm Arrival-Tag {int(idx)}")

### 2) Simulation für Tag 1

In [ ]:
day_start = 1440
day_end = day_start + 1440

filt = (df_plan1["Start"] >=  day_start) & (df_plan1["Start"] < day_end)
df_schedule_1 = df_plan1[filt]
df_schedule_1

In [ ]:
simulation = ProductionDaySimulation(df_schedule_1, vc=0.25)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
show.plot_gantt_machines(df_execution, duration_column = "Simulated Processing Time")

### 3) Verbleibene Jobs nach Tag 1

In [ ]:
df_plan = df_plan1

df_rest = df_plan.merge(df_execution[["Job", "Machine"]], on=["Job", "Machine"], how="left", indicator=True)
df_rest = df_rest[df_rest["_merge"] == "left_only"].drop(columns=["_merge"])
df_rest = df_rest.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_rest

In [ ]:
jssp_data_rest = convert.get_jssp_from_schedule(df_rest)
show.print_jobs(jssp_data_rest)

In [ ]:
df_times = df_times1

allowed = df_rest["Job"].unique() 
df_time_rest = df_times[df_times["Job"].isin(allowed)].copy()
df_time_rest

## Day 2

In [ ]:
jssp_data2, df_times2 = rp.update_new_day(jssp_data_rest, df_time_rest, instance, u_b_mmax= 0.75, generate_deadlines = True, deadlines_buffer_factor= 1.9)
show.print_jobs(jssp_data2)
df_times2

### 1) Rescheduling

In [ ]:
# Reduktion der bereits ausgeführten Operationen auf die relevanten für die nächsten Tage
df_execution_important = df_execution[df_execution["End"] >= 2880]
df_execution_important

In [ ]:
df_plan2 = re_ssv.solve_jssp_weighted_with_fixed_operations_fast(
    job_dict=jssp_data2,
    df_arrivals=df_times2,
    df_executed=df_execution_important,
    solver_time_limit=60**minutes,
    epsilon=0.6,
    arrival_column="Arrival",
    reschedule_start=2880
)

In [ ]:
print(f"Final End:{max(df_plan2.End)}")
df_plan2

In [ ]:
df = df_plan2

block = 1440                                        
for idx, part_df in df.groupby(df['Arrival'] // block, sort=True):
    show.plot_gantt_machines(part_df, f"Gantt-Diagramm - Block {int(idx)}")

### 2) Simulation für Tag 2

In [ ]:
day_start = 2880
day_end = day_start + 1440

filt = (df_plan2["Start"] >=  day_start) & (df_plan2"Start"] < day_end)
df_schedule_2 = df_plan2[filt]
df_schedule_2

In [ ]:
simulation = ProductionDaySimulation(df_schedule_2, vc=0.25)
df_execution, df_undone = simulation.run(start_time = day_start, end_time=day_end)

In [ ]:
show.plot_gantt_machines(df_execution, duration_column = "Simulated Processing Time")

### 3) Verbleibene Jobs nach Tag 2

In [ ]:
df_plan = df_plan2

df_rest = df_plan.merge(df_execution[["Job", "Machine"]], on=["Job", "Machine"], how="left", indicator=True)
df_rest = df_rest[df_rest["_merge"] == "left_only"].drop(columns=["_merge"])
df_rest = df_rest.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_rest

In [ ]:
jssp_data_rest = convert.get_jssp_from_schedule(df_rest)
show.print_jobs(jssp_data_rest)

In [ ]:
df_times = df_times1

allowed = df_rest["Job"].unique() 
df_time_rest = df_times[df_times["Job"].isin(allowed)].copy()
df_time_rest

## Save

In [ ]:
df_plan0.to_csv("data/08a_plan0.csv", index = False)
df_plan1.to_csv("data/08a_plan1.csv", index = False)
df_plan2.to_csv("data/08a_plan2.csv", index = False)